In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
import datetime
from matplotlib import pyplot as plt

%matplotlib inline
%run ./data_loading.ipynb

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=0)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=10*10*12, out_features=300)
        self.fc2 = nn.Linear(in_features=300, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=9)
        
    def forward(self, X):
        X = self.relu(self.conv1(X))
        X = self.pool(self.relu(self.conv2(X)))
        X = self.relu(self.conv3(X))
        X = self.pool(self.relu(self.conv4(X)))
        X = X.view(-1, 10*10*12)
        X = self.relu(self.fc1(X))
        X = self.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [3]:
def calculate_accuracy(dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return acc

In [4]:
def train(dataset, epochs, batch_size=5, to_cuda=True, calculate_acc=False):
    train_acc = []
    val_acc = []
    if to_cuda:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        for i, data in enumerate(dataloader, 0):
            inputs, labels = data
            if to_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = loss_f(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('epoch:', epoch, ', loss:', running_loss * batch_size / dataloader.__len__())
        if calculate_acc:
            t = calculate_accuracy(dataloader)
            v = calculate_accuracy(val_loader)
            print("train acc:", t, "val acc:", v)
            train_acc.append(t)
            val_acc.append(v)
        if epoch % 1 == 0: # every x epochs reshuffle data by creating a new loader
            if to_cuda:
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
            else:
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_acc[-1], val_acc[-1]

In [5]:
cr_test = ClassroomDataset(range_10=[7,10])
# Dividing data into 50% train, 40% val, 30% test
hyper_scores = {}
for wd in [0.0001, 0.001, 0.01, 0.1]:
    scores = []
    for i in range(1):
        net = Net().float()
        if torch.cuda.is_available():
            net = net.cuda()
        loss_f = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=wd)
        if i == 0:
            cr_train = ClassroomDataset(range_10=[0,5])
            cr_val = ClassroomDataset(range_10=[5,7])
        else:
            cr_train = ClassroomDataset(range_10=[2,7])
            cr_val = ClassroomDataset(range_10=[0,2])
        val_loader = DataLoader(cr_val, batch_size=1, shuffle=True)
        train_accuracy, val_accuracy = train(cr_train, to_cuda=False, epochs=20, calculate_acc=True)
        print(train_accuracy, val_accuracy)
        scores.append((train_accuracy, val_accuracy))
    hyper_scores[wd] = scores

  0%|                                                                                   | 0/20 [00:00<?, ?it/s]

epoch: 0 , loss: 10.05963523616083
train acc: 62.03125 val acc: 59.84375


  5%|███▊                                                                       | 1/20 [00:19<06:19, 20.00s/it]

epoch: 1 , loss: 4.0547609763052606
train acc: 69.125 val acc: 66.25


 10%|███████▌                                                                   | 2/20 [00:37<05:46, 19.27s/it]

...

epoch: 18 , loss: 10.709505533799529
train acc: 12.5625 val acc: 12.34375


 95%|██████████████████████████████████████████████████████████████████████▎   | 19/20 [06:38<00:21, 21.34s/it]

epoch: 19 , loss: 10.709252683445811
train acc: 12.5625 val acc: 12.34375


100%|██████████████████████████████████████████████████████████████████████████| 20/20 [06:59<00:00, 21.34s/it]


12.5625 12.34375


In [6]:
hyper_scores

{0.0001: [(100.0, 93.125)],
 0.001: [(99.9375, 92.5)],
 0.01: [(98.375, 90.703125)],
 0.1: [(12.5625, 12.34375)]}

In [7]:
def class_wise_accuracy(dataloader):
    correct = {}
    total = {}
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            for l in range(len(labels)):
                label = int(labels[l].cpu().numpy())
                pred = predicted[l]
                if label in correct:
                    correct[label] += int(pred == label)
                    total[label] += 1
                else:
                    correct[label] = int(pred == label)
                    total[label] = 1
    acc = {}
    for label in correct:
        acc[label] = 100 * correct[label] / total[label]
    return acc

In [8]:
class_wise_accuracy(val_loader)

{8: 0.0, 2: 0.0, 6: 0.0, 5: 0.0, 4: 0.0, 7: 0.0, 3: 0.0, 1: 100.0}